In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
% matplotlib inline

In [2]:
np.random.seed(1000)
def gen_sn(M, I):
    sn = np.random.standard_normal((M+1, I/2))
    sn = np.concatenate((sn, -sn), axis=1)
    sn = (sn - sn.mean()) / sn.std()
    return sn

In [3]:
def gbm_mcs_amer(K, option='call'):
    #Time step
    dt = T/M
    #discount factor
    df = np.exp(-r*dt)
    #Simulation of index level
    S = np.zeros((M+1, I))
    S[0] = S0
    sn = gen_sn(M,I)
    for t in range(1, M+1):
        S[t] = S[t-1] * np.exp((r-0.5*sigma**2) * dt + sigma*np.sqrt(dt)*sn[t])
    
    #Case base simulation payoff
    if option == 'call':
        h = np.maximum(S-K, 0)
    else:
        h = np.maximum(K-S, 0)
        
    #LSM algorithm
    V = np.copy(h)
    #Iterate from  M-1 to 0
    for t in range(M-1, 0, -1):
        #Least-square regression to estimate V[t+1]*df based on S[t], level 7 of the polynomial
        reg = np.polyfit(S[t], V[t+1]*df, 7)
        if t == 30:
            reg_acu = reg
        # For the coefficients obtained, evaluate reg*S[t] to obtain an expectaction of the continuation value C
        C = np.polyval(reg, S[t])
        #Compute for each time stamp t, the value of the american option, as the maximum between the expected continuation value
        # and the payoff h at t
        V[t] = np.where(C>h[t], V[t+1]*df, h[t])
        
    #MCS estimator
    C0 = df * 1/I * np.sum(V[1])
    return C0, V, h, sn, S, reg, C, reg_acu

In [4]:
M=50
I=50000
S0 = 100
r = 0.05
sigma = 0.25
T = 1

In [5]:
C0, V, h, sn, S, reg, C, reg_acu = gbm_mcs_amer(110., option='call')

In [6]:
S

array([[ 100.        ,  100.        ,  100.        , ...,  100.        ,
         100.        ,  100.        ],
       [  97.53284455,   92.18118931,   87.62200211, ...,  101.01427171,
         103.237074  ,   98.55789431],
       [  96.82599963,   95.46681946,   89.71343461, ...,   97.32077671,
         111.56238641,   96.83867064],
       ..., 
       [  79.13245316,  114.3061148 ,   84.87802538, ...,  101.78766767,
         122.71767645,  120.49993539],
       [  75.53313939,  119.72277134,   89.81675334, ...,  103.39240353,
         125.13986626,  117.8917837 ],
       [  75.67523521,  121.95856592,   91.12880713, ...,  112.53447269,
         132.70432859,  117.23937521]])

In [ ]:
V.shape

In [7]:
S[-1]

array([  75.67523521,  121.95856592,   91.12880713, ...,  112.53447269,
        132.70432859,  117.23937521])

In [ ]:
h[-1]

In [ ]:
plt.plot(S[:, :1000], alpha=0.2);
plt.grid(True)

In [ ]:
plt.plot(h[:, :1000], alpha=0.2)
plt.grid(True);

In [ ]:
plt.plot(V[:, :1000], alpha=0.2);
plt.grid(True)

In [ ]:
V.shape